## Reinforcement Learning for SMS Messaging to Improve Medication Adherence - Roybal

In [9]:
import sys
import time
import dateutil
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankRequest
from msrest.authentication import CognitiveServicesCredentials
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime
from collections import Counter
import string
import pickle
import json
import pytz
import os
import re

In [10]:
from patient_data import import_pt_data,export_pt_data, new_empty_pt_data
from pillsy_parser import import_Pillsy, find_rewards
from driverReward import get_reward_update,send_rewards
from redcap_parser import import_redcap, update_pt_data_with_redcap
from driverRank import run_ranking, write_sms_history
from control_disconnection import check_control_disconnectedness

### Start Program Timer

In [11]:
# run_time = datetime.now()
# testing_flag = input("Testing with another date? y/n: ")
# if testing_flag.lower() == "y":
#     print("Set testing run time: ")
#     run_time_yyyy_mm_dd_input = input("Enter the testing date: YYYY-MM-DD ")
#     timestamp = "10:30 AM " + run_time_yyyy_mm_dd_input
#     run_time = dateutil.parser.parse(timestamp)
# run_time = pytz.timezone("America/New_York").localize(run_time)
# run_time

run_time = pytz.timezone("America/New_York").localize(dateutil.parser.parse("10:30 AM 2020-11-23"))
run_time

datetime.datetime(2020, 11, 23, 10, 30, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

### Set Up MS Azure Personalizer Client

Defining and Instantiating a Personalizer Client:

Personalizer Keys:
* In the Microsoft Azure Dashboard, navigate to our bwh-pharmacoepi-roybal-dev-use2-cog Cognitive Services page.
* Within the Keys and Endpoint section, copy either Key 1 or Key 2 to enter as the Personalizer Key.

Personalizer Endpoint:
* https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/

In [12]:
with open(os.path.join("..", ".keys", "azure-personalizer-key.txt"), 'r') as f:
     personalizer_key = f.read().rstrip()
personalizer_endpoint = "https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/"
client = PersonalizerClient(personalizer_endpoint, CognitiveServicesCredentials(personalizer_key))

## Reward Step

If we've already initiated the trial, we will have:
* Pre-existing patient dataset in need of reward updates
* Pillsy data from yesterday to determine reward
If this is study initiation, this step will just load an empty patient dictionary and null pillsy dataset.

In [13]:
pt_data = import_pt_data(run_time)
new_pillsy_data = import_Pillsy(run_time)

if not pt_data.empty and not new_pillsy_data.empty:

    # From Pillsy data, computes the Rewards to send to Personalizer for each patient's Rank calls from yesterday's run.
    pt_data = find_rewards(new_pillsy_data, pt_data, run_time)

    # using updated patient data (new pillsy + patient data), format the rewards to Personalizer into a dataframe
    rewards_to_send = get_reward_update(pt_data, run_time)

    # actual call to personalizer
    send_rewards(rewards_to_send, client)
    export_pt_data(pt_data, run_time, "reward")

## Import/Update Patients

In [14]:
redcap_data = import_redcap(run_time)
pt_data = update_pt_data_with_redcap(redcap_data, pt_data, run_time)

## Rank Step
Call Personalizer to rank action features to find the correct text message to send today.

In [15]:
ranked_pt_data = new_empty_pt_data()
for index, patient in pt_data.iterrows():
    if patient["censor"] != 1 and patient["censor_date"] > run_time.date():
        patient = run_ranking(patient, client, run_time)
        ranked_pt_data = ranked_pt_data.append(patient)
        #need checks if pass by ref or val
export_pt_data(ranked_pt_data, run_time, "rank") # log

rankresult 1 1 0 1 1
    factor_set  text_number  \
70          26           71   
71          26           72   
72          26           73   

                                         text_message  framing_sms  \
70  Did you know you took your medicine X out of t...            1   
71  In the last week, you took your medicine X day...            1   
72  How do you think you are doing with your daily...            1   

    history_sms  social_sms  content_sms  reflective_sms  quantitative_sms  \
70            1           0            1               1                 0   
71            1           0            1               1                 0   
72            1           0            1               1                 0   

    doctor_sms  lifestyle_sms  
70           0              1  
71           1              0  
72           0              0  
rankresult 0 0 1 0 1
    factor_set  text_number  \
9            4           10   
10           4           11   
11           4    

## Output SMS and Patient Data

In [16]:
write_sms_history(ranked_pt_data, run_time)
export_pt_data(ranked_pt_data, run_time, "final") # input for tomorrow
check_control_disconnectedness(run_time) # check whether controls have connection problems

To Do List:
  * Executability by RA / User friendliness
    * [x] - Lily done by using Jupyter notebook - Making this more user friendly than a  Command Line
    * [ ] - JOE TODO / help brainstorm - make jupyter notebook pretty and write a how to document for new non CS person to be able to execute this from Jupyter notebook
    * [ ] - JOE TODO / help brainstorm - Making this more user friendly than a Jupyter Notebook - to do by doing a main.py executable bash script
    * [ ] - JOE TODO / help brainstorm - Hooks into Pillsy/RedCap for data retrieval - need check in with constance for pillsy and ellen for redcap
    * [ ] - JOE TODO / help brainstorm - (probably not feasible) Hooks into SMS Platform to automate text sending - need check in with constance
    * [ ] - JOE TODO / help brainstorm - how to handle the definite 1 human entered variable of personalizer key - file to direct to or? - maybe as marco/elad
    * [ ] - Lily/JOE TODO / help - try to break this code in any way possible
    * [ ] - Lily/JOE TODO / help - debug, unit testing
    * [ ] - Lily/JOE TODO / help - run it fully several times based in dropbox together
    * [ ] - Lily/JOE TODO / help - Make a log file that will report high level information from running this like a run summary
        * elements to include:
            * start and end time of the run
            * how many patients were read in from each import statement
            * how many reward calls were successfull made
            * how many rank calls were successfull made
            * any other meta data that will help us debug and ensure this is all working as planned


